<a href="https://colab.research.google.com/github/ajraymond27/DeepLearningVids/blob/master/Fast_Style_NST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Enabling and Testing GPU**

In [ ]:
#@title Hit This GPU Bruh
#Check if GPU is connected
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# How much GPU, tho?
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# What about RAM?
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# **Import & Configure**

In [ ]:
#@title Art Vandalay, Importer / Exporter
import os
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
tf.__version__

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools
import time

# **Set Path to Content and Style Images**

In [ ]:
#@title Download Google
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [ ]:
#@title Authenticate Yourself
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#@title Search for your .zip file in company Google Drive folder
#@markdown Format the two fields like this (copy/paste):

#@markdown query: title = 'example.zip'

#@markdown zip: example.zip

query = "title='example.zip'" #@param {type:"string"}
zip = "example.zip" #@param {type:"string"}

# fid = drive.ListFile({'q':"title='weed.zip'"}).GetList()[0]['id']
# f = drive.CreateFile({'id': fid})
# f.GetContentFile('weed.zip')

fid = drive.ListFile({'q': query}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile(zip)

In [ ]:
#@title Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# **Load Content and Style Images**

## Unzip your .zip
Change "example.zip" to your zip file name. The code below should look like this:


---



f.keys()

!unzip example.zip


In [ ]:
f.keys()
!unzip example.zip

In [ ]:
#@title Download all content and style images
PATH = '/content/' + zip.strip(".zip")

# os.remove(os.path.join(PATH,
#                          'content/.DS_Store'))

os.listdir(PATH)

content_path = os.path.join(PATH,
                         'content/')

style_path = os.path.join(PATH,
                        'style/style.png')

def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
  max_dim = 3840
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

content_images = []

# iterate through the names of contents of the folder
for image_path in sorted(os.listdir(content_path)[0:5]):

  # create the full input path and read the file
  input_path = os.path.join(content_path, image_path)
  content_image = load_img(input_path)

  content_images.append(content_image)


style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_images[0], 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

# Fast Style Transfer

In [ ]:
#@title Run Fast Style Transfer
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

fast_styled_images = []
start = time.time()

for i in content_images[0:2]:
  stylized_image = hub_model(tf.constant(i), tf.constant(style_image))[0]
  tensor_to_image(stylized_image)
  fast_styled_images.append(stylized_image)

end = time.time()
print("Total time: {:.1f}".format(end-start))
print("Number of Styled Frames: {}".format(len(fast_styled_images)))
print("Example:")
tensor_to_image(fast_styled_images[0])

# Save Styled Images

In [ ]:
#@title Save output to company drive folder
import os
os.chdir('/content/drive/MyDrive/entertAIn/Output/' + zip)
!pwd

count = 0

for i in fast_styled_images:
  file_name = 'fast-styled-image-' + str(count) + ".png"
  tensor_to_image(i).save(file_name)
  files.download(file_name)
  count += 1